In [22]:
import requests
import pandas as pd
import json
import psycopg2
from sqlalchemy import create_engine


In [20]:
def save_table(df,name):
    import io
    df.head(0).to_sql(f'{name}', engine, if_exists='replace',index=False) #truncates the table

    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, f'{name}' , null="") # null values become ''
    conn.commit()
    return

In [23]:
engine = create_engine('postgresql+psycopg2://username:secret@db:5432/database')
conn = psycopg2.connect(database="database",user="username", password="secret",host="db", port="5432")
cur = conn.cursor()

In [ ]:
data = 'https://api.covidtracking.com/v1/states/daily.json'
d =requests.get(data)
df = pd.read_json(d.text)


In [24]:
save_table(df,'daily_covid')

In [26]:
#https://github.com/nytimes/covid-19-data
data = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
df = pd.read_csv(data)

In [ ]:
save_table(df,'base_covid')

In [40]:
#docs https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf
data = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/cc-est2019-alldata.csv'
df = pd.read_csv(data,encoding='ISO-8859-1')
df = df.query('YEAR==12')
save_table(df,'population')

In [16]:
data = 'https://overflow.solutions/wp-content/uploads/2020/05/Poverty_County_18.csv'
df = pd.read_csv(data)

,Geo_Name,Total_Pop,Child_Pop,Elder_Pop,Pov_Rate,Pov_Rate_Child,Pov_Rate_Elder,State_FIPS,County_FIPS
0,"Washington County, Mississippi",47086,12349,6953,33.3,48.1,17.1,28,151
1,"Perry County, Mississippi",12028,2809,2072,18.7,25.6,16.6,28,111
2,"Choctaw County, Mississippi",8321,1844,1740,23.3,38.3,13.1,28,19
3,"Itawamba County, Mississippi",23480,5134,4005,15.4,18.3,13.0,28,57
4,"Carroll County, Mississippi",10129,1969,2249,14.8,18.1,14.9,28,15


In [21]:
save_table(df,'poverty')

In [45]:
#https://overflow.solutions/demographic-traits/education/what-u-s-counties-are-the-most-educated/
data = 'https://overflow.solutions/wp-content/uploads/2020/04/Educ_Attain_County_18.csv'
df = pd.read_csv(data)

In [46]:
save_table(df,'education')

In [4]:
import requests
#website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
save_table(df,'income')